# Extract 2D Area Cells from HEC-RAS .HDF files for all plans in a directory

Created on Wed Aug 19 13:47:47 2015
@author: mwarms

This code can be used to extract data from HEC-RAS 2D area cells of interest for all plans--specifically the maximum water surface. Useful when there are numerous plans--in this case an incremental hazard analysis was performed, so ~50 plans were run overnight. 

In [21]:
import pandas as ps # useful databases/arrays for storing data
import h5py         # read HEC-RAS HDF files (pandas HDFStore does not work)           
import os as os     # windows operating system  

In [22]:
# initialize data frame in which to store data (in this case we want maximum WSE and time):
MaxWSE = ps.DataFrame()
MaxWSEtime = ps.DataFrame()  # <-- this is unnecessary if you do not want the time at which the peak occurs

In [23]:
# initialize path variable:
path = "C:\users\mwarms\documents\USSD Conference 2016\RAS HDFs"


In [24]:
# initialize HDFkey (i.e. path to desired variable in HDF file):
    # to determine the key, you can either view the HDF manually or open the HDF
    # using h5py (see below) and use the h5py.keys() function (e.g. f_temp.keys())

example_h5py_file = h5py.File(path + "\NineMileDamBreach.p01.hdf")
example_h5py_file.keys()

[u'Event Conditions', u'Geometry', u'Plan Data', u'Results']

In [25]:
example_h5py_file['Results'].keys()

[u'Summary', u'Unsteady']

In [26]:
example_h5py_file['Results/Unsteady'].keys()

[u'Geometry Info', u'Output', u'Summary']

In [27]:
# continue this logic until you reach the key that you want. Again, this can be done in python or viewing the HDF file manually.

HDFkey = '/Results/Unsteady/Output/Output Blocks/Base Output/Summary Output/2D Flow Areas/Long Lake & L.S./Maximum Water Surface'

In [28]:
# initialize HEC-RAS project name:
HECRASProject = "NineMileDamBreach"

In [29]:
# initialize total number of 2D area cells (array 0 = max wse data, 1 = time which max occurs), both will be the same size: 
TotNumCells = example_h5py_file[HDFkey][0].size
example_h5py_file.close()
TotNumCells

8912

In [34]:
# Loop through HDF plan files and extract Max WSE for all cells
os.chdir(path)  # <--necessary when your ipynb is not in the same folder as the data you care about

i = 0
for file in os.listdir(path):
    if file.endswith(".hdf") and file.startswith(HECRASProject + ".p"):
        f_temp = h5py.File(file)
        d_temp = f_temp[HDFkey][:]    
        MaxWSE[i] = ps.Series(d_temp[0,0:TotNumCells])
        MaxWSEtime[i] = ps.Series(d_temp[1,0:TotNumCells])
        i+=1 

In [36]:
# initialize variable for total number of plans
TotalPlans = i
TotalPlans

60

In [37]:
# Which cells do we care about in the 2D Area? (Upstream to Downstream)
SelectedCells = [5036,3996,3695,3421,3260,2923,2485,2056,1542,930,855,784,
                 590,366,313,403,448,614,943,1480,1990,2086,1976,1870,1555] 

In [38]:
# HEC-RAS plan numbers belonging to each geometry (13 excluded--not used)
#    Note: This is only necessary if you have multiple geometries 
#    Normally, you could use this line:
#    Plans = range(1,TotalPlans+1)
HighScourPlans = sorted([16,2,5,4,9,8,6,12,11,10,15,14])
MedScourPlans = sorted([17,3,18,20,22,24,26,28,30,32,34,36])
LowScourPlans = sorted([1,7,19,21,23,25,27,29,31,33,35,37])
IHAPlans = [13,38,39,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]


In [39]:
# Query MaxWSE data to extract just the cells we are interested in. Again, 
#    since we have 3 different geometries, the selected cells change depending
#    on the plan
CellsMaxWSEHighScour = ps.DataFrame()  # initialize data frame in which to store data
CellsMaxWSEMedScour = ps.DataFrame()   # initialize data frame in which to store data
CellsMaxWSELowScour = ps.DataFrame()   # initialize data frame in which to store data
CellsMaxWSEIHA = ps.DataFrame()        # initialize data frame in which to store data

In [41]:
# Loop through plans to populate the different dataframes (mult. by 24 to convert to hours)
for j in range(1,TotalPlans+1):
    if j in HighScourPlans:
        CellsMaxWSEHighScour[0,j] = MaxWSE[j-1].iloc[SelectedCells[:]]
        CellsMaxWSEHighScour[1,j] = MaxWSEtime[j-1].iloc[SelectedCells[:]]*24
    elif j in MedScourPlans:
        CellsMaxWSEMedScour[0,j] = MaxWSE[j-1].iloc[SelectedCells[:]]
        CellsMaxWSEMedScour[1,j] = MaxWSEtime[j-1].iloc[SelectedCells[:]]*24
    elif j in LowScourPlans:
        CellsMaxWSELowScour[0,j] = MaxWSE[j-1].iloc[SelectedCells[:]]
        CellsMaxWSELowScour[1,j] = MaxWSEtime[j-1].iloc[SelectedCells[:]]*24
    elif j in IHAPlans:
        CellsMaxWSEIHA[0,j] = MaxWSE[j-1].iloc[SelectedCells[:]]
        CellsMaxWSEIHA[1,j] = MaxWSEtime[j-1].iloc[SelectedCells[:]]*24

In [42]:
# Write Extracted output to excel spreadsheet or csv:
  # Excel example (not used) below:   
  #    writer = ps.ExcelWriter('IncrementalHazardAnalysis.xlsx')
  #    CellsMaxWSEHighScour.to_excel(writer,'HighScourPlans')
  #    CellsMaxWSEMedScour.to_excel(writer,'MedScourPlans')
  #    CellsMaxWSELowScour.to_excel(writer,'LowScourPlans')
  #    writer.save()
CellsMaxWSEHighScour.to_csv('HighScour.csv', sep=',')
CellsMaxWSEMedScour.to_csv('MedScour.csv', sep=',')
CellsMaxWSELowScour.to_csv('LowScour.csv', sep=',')
CellsMaxWSEIHA.to_csv('IHA.csv', sep=',')